In [ ]:
import torch
import random
import os
from diffusers import DiffusionPipeline
from diffusers import StableDiffusionPipeline, EulerDiscreteScheduler
import torch
from pipeline_rf import RectifiedFlowPipeline
from diffusers import AutoPipelineForText2Image
from dataclasses import dataclass
from generate import generate_single_image, load_model

# config

In [1]:
@dataclass
class TestConfig:
    device = "cuda"
    save_path = "/data/liutao/AFHQ/"
    save_path_other = "/data/liutao/FFHQ/"
    sd_id = "/data/model/stable-diffusion-2-1-base"
    instaflow_id = "/data/model/instaflow_0_9B_from_sd_1_5"
    lcm_id = "/data/model/LCM_Dreamshaper_v7"
    sdxl_turbo_id = "/data/model/sdxl-turbo"
    ours_path = "/data/"
    ours_model_id = "/data/20231212/SwiftBrush_reproduce_se_parallel/checkpoints_20240228/vsd_global_step38000_8nis.pth"
    # test_prompt = ["A small waterfall in the middle of rocks, an airbrush painting", "A oil painting of red roses in a blue vase", "A brown and white dog running through water",
    #                 "Reflection of the glass cube building and water surface", "A butterfly on flowers", "Milk and a sandwich with knife on a table", "A cute cat", "A cute dog",
    #                  "A group of mushrooms that are sitting on the ground", "A man with a beard wearing glasses and a hat in blue shirt"]
    # # test_prompt_other = []
    test_prompt = ["A real high-quality cat face", "A real high-quality dog face", "A real high-quality wild animal face"]
    test_prompt_other = ["a photo of man","a photo of woman"]
    items = ["lcm", "sdxl_turbo"] # options: ["lcm","sdxl_turbo","instaflow","sd"]
    ours_item = "ours_38k"
    sd_steps = [50]
    instaflow_steps = [1]
    lcm_steps = [1, 4]
    sdxl_turbo_steps = [1, 4]
    ours_steps = [8]
    resolution = 512
    start_sample_num = 1
    end_sapmle_num = 1000
    file_extension = ".jpg"
config = TestConfig()

NameError: name 'dataclass' is not defined

In [ ]:
if not os.path.exists(config.save_path):
    os.makedirs(config.save_path)
if not os.path.exists(config.save_path_other):
    os.makedirs(config.save_path_other)
        
# Iterate over the list with different steps
for item in config.items:
    if item == "sd":
        steps = config.sd_steps
        scheduler = EulerDiscreteScheduler.from_pretrained(config.sd_id, subfolder="scheduler")
        pipe = StableDiffusionPipeline.from_pretrained(config.sd_id, scheduler=scheduler, torch_dtype=torch.float16)
    elif item == "lcm":
        steps = config.lcm_steps
        pipe = DiffusionPipeline.from_pretrained(config.lcm_id, safety_checker=None, requires_safety_checker=False, torch_dtype=torch.float16)
    elif item == "sdxl_turbo":
        steps = config.sdxl_turbo_steps
        pipe = AutoPipelineForText2Image.from_pretrained(config.sdxl_turbo_id, torch_dtype=torch.float16, variant="fp16")
    elif item == "instaflow":
        steps = config.instaflow_steps
        pipe = RectifiedFlowPipeline.from_pretrained(config.instaflow_id, torch_dtype=torch.float16, safety_checker=None, requires_safety_checker=False) 
    pipe = pipe.to(config.device)
    pipe.set_progress_bar_config(disable=True)
    for step in steps:
        print(f"{item}:{step}")
        dir_name = item+str(step)
        directory = os.path.join(config.save_path,dir_name)
        if not os.path.exists(directory):
            os.makedirs(directory)
        
        directory_other = os.path.join(config.save_path_other,dir_name)
        if not os.path.exists(directory_other):
            os.makedirs(directory_other)

        for prompt in config.test_prompt:
            prompt_dir = os.path.join(directory,prompt)
            if not os.path.exists(prompt_dir):
                os.makedirs(prompt_dir)
        for i in range(config.start_sample_num, config.end_sapmle_num):
            images = pipe(prompt=config.test_prompt, num_inference_steps=step, width=config.resolution, height=config.resolution).images
            for index, prompt in enumerate(config.test_prompt):
                prompt_dir = os.path.join(directory,prompt)
                image_name = prompt+"_"+str(i)+config.file_extension
                images[index].save(os.path.join(prompt_dir,image_name))
        
        for prompt in config.test_prompt_other:
            prompt_dir_other = os.path.join(directory_other,prompt)
            if not os.path.exists(prompt_dir_other):
                os.makedirs(prompt_dir_other)
        for i in range(config.start_sample_num, config.end_sapmle_num):
            images = pipe(prompt=config.test_prompt_other, num_inference_steps=step, width=config.resolution, height=config.resolution).images
            for index, prompt in enumerate(config.test_prompt_other):
                prompt_dir_other = os.path.join(directory_other,prompt)
                image_name = prompt+"_"+str(i)+config.file_extension
                images[index].save(os.path.join(prompt_dir_other,image_name))

    torch.cuda.empty_cache()    
        

In [ ]:
item = config.ours_item
steps = config.ours_steps
vae, tokenizer, text_encoder, unet, scheduler, alphas = load_model(config.ours_path, config.ours_model_id, config.device)
for step in steps:
    print(f"{item}:{step}")
    dir_name = item+str(step)
    directory = os.path.join(config.save_path,dir_name)
    if not os.path.exists(directory):
        os.makedirs(directory)
    
    directory_other = os.path.join(config.save_path_other,dir_name)
    if not os.path.exists(directory_other):
        os.makedirs(directory_other)

    for prompt in config.test_prompt:
        prompt_dir = os.path.join(directory,prompt)
        if not os.path.exists(prompt_dir):
            os.makedirs(prompt_dir)
        for i in range(config.start_sample_num, config.end_sapmle_num):
            image = generate_single_image(network=(vae, tokenizer, text_encoder, unet, scheduler),prompt=prompt, seed=random.randint(1, 1e11),num_inference_steps=step)
            image_name = prompt+"_"+str(i)+config.file_extension
            image.save(os.path.join(prompt_dir,image_name))
        print(prompt)

    for prompt in config.test_prompt_other:
        prompt_dir_other = os.path.join(directory_other,prompt)
        if not os.path.exists(prompt_dir_other):
            os.makedirs(prompt_dir_other)
        for i in range(config.start_sample_num, config.end_sapmle_num):
            image = generate_single_image(network=(vae, tokenizer, text_encoder, unet, scheduler),prompt=prompt, seed=random.randint(1, 1e11),num_inference_steps=step)
            image_name = prompt+"_"+str(i)+config.file_extension
            image.save(os.path.join(prompt_dir_other,image_name))
        print(prompt)
    
torch.cuda.empty_cache()